In [24]:
import os
import json
import shutil
from tqdm import tqdm
import pandas as pd
from paddlenlp import Taskflow

In [2]:
import os
from tqdm import tqdm
import pandas as pd
from paddlenlp import Taskflow
import json

class DocVQA(): 

    def __init__(self): 

        # Get paddlenlp taskflow pipeline
        self.docprompt = Taskflow("document_intelligence", lang="en")
        
    def prompt_question(self, image_path, questions, doc_type="image"): 

        output_path = "/data/app/fast_api/qa_model/pdf_image_{}.jpeg"
        if doc_type == "pdf": 
            
            image_query_list = list()
            doc = fitz.open(image_path)
            for page_id, page in enumerate(doc): 
                
                print('processing page ', page_id)
                pix = page.get_pixmap()
                pix.save(output_path.format(page_id))

                query = {
                    "doc" : output_path.format(page_id), 
                    "prompt": [ que for que in questions if que != ""]
                }
                encoding = self.docprompt([query])

                image_query_list.append(encoding)
            
            response = get_max_prob_prompt(image_query_list)

        elif doc_type == "image": 

            query = {
                "doc" : image_path, 
                "prompt": [ que for que in questions if que != ""]
            }

            response = self.docprompt([query])

        return response

obj = DocVQA()
    
# Define the input folder containing image files and the CSV output file path
input_folder = '/data/nha_phase_1/images/discharge_summary'
output_csv_file = '/data/user/nevil/results/clustering/hospitalwise/hospital_name_file_wise.csv'

# Define a list of questions
questions = ["What is name of the hospital?", "What is hospital address?", "What is the state of the hospital?"]  

# Initialize an empty list to store the results
results = []

# Loop through the image files in the input folder
i = 0
for filename in tqdm(os.listdir(input_folder)):
    
    if i == 100: break
    i+=1    
    
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):  
        image_path = os.path.join(input_folder, filename)
        
        # Call the prompt_question() function with image_path and questions
        json_output = obj.prompt_question(image_path, questions)
        
        # Append the result to the results list
        results.append({'Image File': filename, 'JSON Output': json_output})

# Create a DataFrame from the results list
df = pd.DataFrame(results)

# Write the DataFrame to a CSV file
df.to_csv(output_csv_file, index=False)

df['raw_hospital_name'] = df['JSON Output'].apply(lambda x: x[0]['result'][0]['value'])
df.to_csv("/data/user/nevil/results/clustering/hospitalwise/hospital_name_file_wise_full.csv", index=False)

[2023-09-17 13:25:11,274] [    INFO] - We are using <class 'paddlenlp.transformers.ernie_layout.tokenizer.ErnieLayoutTokenizer'> to load 'ernie-layoutx-base-uncased'.
[2023-09-17 13:25:11,275] [    INFO] - Already cached /home/nhadmin/.paddlenlp/models/ernie-layoutx-base-uncased/vocab.txt
[2023-09-17 13:25:11,276] [    INFO] - Already cached /home/nhadmin/.paddlenlp/models/ernie-layoutx-base-uncased/sentencepiece.bpe.model
[2023-09-17 13:25:11,989] [    INFO] - tokenizer config file saved in /home/nhadmin/.paddlenlp/models/ernie-layoutx-base-uncased/tokenizer_config.json
[2023-09-17 13:25:11,999] [    INFO] - Special tokens file saved in /home/nhadmin/.paddlenlp/models/ernie-layoutx-base-uncased/special_tokens_map.json
  2%|▏         | 100/5385 [14:09<12:28:35,  8.50s/it]


,Image File,JSON Output,raw_hospital_name
0,1581143296927_discharge summary_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",& BEHGALHOSPITAI
1,1581143296927_discharge summary_p1.jpeg,"[{'prompt': 'What is name of the hospital?', '...",
2,1581143296927_discharge summary_p2.jpeg,"[{'prompt': 'What is name of the hospital?', '...",Behgalancer Centre&Hospltal
3,1581329810948_DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",Amar Hospital
4,1581329367321_Discharge summary_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...","HIMACHAL PRADESHRegional Hospital, Kalle"
...,...,...,...
95,1580707287898_SAVITA DISCHARGE SUMMARY.PDF_p1....,"[{'prompt': 'What is name of the hospital?', '...",SAI SANJIVNI HOSPITAL
96,1580552416935_discharge summary.PDF_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",CAPITOLHOSPITAL
97,1580552416935_discharge summary.PDF_p1.jpeg,"[{'prompt': 'What is name of the hospital?', '...",CAPITOLHOSPITAL
98,1580552416935_discharge summary.PDF_p2.jpeg,"[{'prompt': 'What is name of the hospital?', '...",CAPITOLHOSPITAL


In [12]:
df.groupby('raw_hospital_name').count()

,Image File,JSON Output
raw_hospital_name,,
,30,30
& BEHGALHOSPITAI,1,1
Amar Hospital,3,3
BAM,1,1
BavtupBD,1,1
Behgalancer Centre&Hospltal,1,1
CAPITOLHOSPITAL,6,6
CONSUETA,1,1
Government Medical College and Hospital,1,1


In [2]:
data_df = pd.read_csv("/data/user/nevil/results/clustering/hospitalwise/3000_hospital_name_file_wise.csv")
print(data_df.shape)
data_df.head()

(2995, 2)


,Image File,JSON Output
0,1659689273798_Patient Discharge Summary Form_p...,"[{'prompt': 'What is name of the hospital?', '..."
1,1659689273798_Patient Discharge Summary Form_p...,"[{'prompt': 'What is name of the hospital?', '..."
2,1659692087280_AB DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '..."
3,1659692087280_AB DISCHARGE SUMMARY_p1.jpeg,"[{'prompt': 'What is name of the hospital?', '..."
4,1659690489208_HOS DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '..."


In [11]:
data_df["JSON Output"] = data_df["JSON Output"].apply(eval)
data_df["raw_hospital_name"] = data_df["JSON Output"].apply(lambda x: x[0]["result"][0]["value"])
data_df

,Image File,JSON Output,raw_hospital_name
0,1659689273798_Patient Discharge Summary Form_p...,"[{'prompt': 'What is name of the hospital?', '...",Nemcare Hospital
1,1659689273798_Patient Discharge Summary Form_p...,"[{'prompt': 'What is name of the hospital?', '...",NA
2,1659692087280_AB DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",HospitaISTERLING RAM KRISHNA SPECIALITY HOSPITAL
3,1659692087280_AB DISCHARGE SUMMARY_p1.jpeg,"[{'prompt': 'What is name of the hospital?', '...",STERUNG
4,1659690489208_HOS DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",Ram Krishna Speciality Hospital
...,...,...,...
2990,1664251980576_DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",WOODLAND HOSPITAL
2991,1664251980576_DISCHARGE SUMMARY_p1.jpeg,"[{'prompt': 'What is name of the hospital?', '...",WOODLAND HOSPITAL
2992,1664251980576_DISCHARGE SUMMARY_p2.jpeg,"[{'prompt': 'What is name of the hospital?', '...",WOODLAND HOSPITAL
2993,1664249648363_DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",DEVKAMALHOSPITAL&RESEARCH CENTREHOSPITAL


In [12]:
data_df.groupby("raw_hospital_name").count(apply)

,Image File,JSON Output
raw_hospital_name,,
,749,749
"""Raisina Hospital""",1,1
& STONE CENTRE,1,1
(FOGSI Himalay Hospita),1,1
*,1,1
...,...,...
sterlingtospital,1,1
tom Krisina Spociality Hospito,1,1
uhack,1,1


In [20]:
print("Unique hospitals: ", data_df["raw_hospital_name"].nunique())

Unique hospitals:  855


In [21]:
data_df["total_report"] = data_df.groupby('raw_hospital_name')['Image File'].transform('count')
data_df = data_df[(data_df["total_report"]>5)]
print(data_df.shape)
data_df

(1899, 4)


,Image File,JSON Output,raw_hospital_name,total_report
1,1659689273798_Patient Discharge Summary Form_p...,"[{'prompt': 'What is name of the hospital?', '...",NA,201
4,1659690489208_HOS DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",Ram Krishna Speciality Hospital,25
5,1659692527815_DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",,749
6,1659669074161_DISCHARGESUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",SAIMS HOSPITALINDORE,8
7,1659669074161_DISCHARGESUMMARY_p1.jpeg,"[{'prompt': 'What is name of the hospital?', '...",,749
...,...,...,...,...
2987,1664249205324_DISCHARGE SUMMARY 26_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",GNRC MEDICAL,53
2989,1664249613824_DISCHARGE SUMMARY 26_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",GNRC MEDICAL,53
2990,1664251980576_DISCHARGE SUMMARY_p0.jpeg,"[{'prompt': 'What is name of the hospital?', '...",WOODLAND HOSPITAL,66
2991,1664251980576_DISCHARGE SUMMARY_p1.jpeg,"[{'prompt': 'What is name of the hospital?', '...",WOODLAND HOSPITAL,66


In [22]:
print("Unique hospitals: ", data_df["raw_hospital_name"].nunique())

Unique hospitals:  61


In [27]:
source_path = "/data/nha_phase_1/images/discharge_summary_new/"
output_base_path = "/data/user/nevil/results/clustering/hospitalwise/images/by_name"

# Iterate through the DataFrame rows
for index, row in data_df.iterrows():
    source_file_path = source_path + row['Image File']
    hospital_name = row['raw_hospital_name']
    
    if hospital_name in ["", "NA"]: continue
        
    # Construct the destination folder path based on 'hospital_name'
    destination_folder = os.path.join(output_base_path, hospital_name)
    
    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)
    
    # Extract the filename from the source path
    filename = os.path.basename(source_file_path)
    
    # Construct the destination file path
    destination_file_path = os.path.join(destination_folder, filename)
    
    # Copy the file to the destination folder
    shutil.copy(source_file_path, destination_file_path)

    print(f"File '{filename}' copied to '{destination_folder}'")


File '1659690489208_HOS DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Ram Krishna Speciality Hospital'
File '1659669074161_DISCHARGESUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/SAIMS HOSPITALINDORE'
File '1659692581012_AB DISCHARGE SUMMARY_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Ram Krishna Speciality Hospital'
File '1659675712625_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/GNRC MEDICAL'
File '1659693888367_DISCHARGE SUMMARY.PDF_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/OODLANDOSPITAL'
File '1659694509500_DISCHARGE SUMMARY.PDF_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/OODLANDOSPITAL'
File '1659694723221_HOS DISCHARGE SUMMARY_p2.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by

File '1663994419871_DISCHARGE SUMMARY DOCUMENT_p5.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/LOK SEVA'
File '1663994419871_DISCHARGE SUMMARY DOCUMENT_p8.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/LOK SEVA'
File '1663990939707_Discharge Summary_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/DR SHROFF'S CHARITY EYE HOSPITAL-MOHAMMADI'
File '1663994412497_DISCHARGE SUMMARY.PDF_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/OODLANDOSPITAL'
File '1663994547536_DISCHARGESUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/MAKARIOS MEDICALCENTER'
File '1663994547536_DISCHARGESUMMARY_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/MAKARIOS MEDICALCENTER'
File '1663994547536_DISCHARGESUMMARY_p6.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name

File '1663999710526_discharge summary_0001_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Santosh Hospital'
File '1663999710526_discharge summary_0001_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Santosh Hospital'
File '1663999717926_DISCHARGE SUMMARY DECLARATION_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/EXCELCAREHOSPITALS'
File '1664000071875_Discharge_Summary_General-4_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/SHRLVINOBA BHAVE CIVILHOSPITAL'
File '1664000077461_SAVITABEN SOFTWARE DISCHARGE SUMMARY  _p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/AMRELI INSTITUTE'
File '1664000155635_DISCHARGE SUMMARY ANC PACKAGE_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Nazareth Hospital'
File '1664000157696_DISCHARGE SUMMARY  DECLARATION_p0.jpeg' copied to '/data/u

File '1664006108712_AB DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/HospitalSTERLING RAM KRISHNA SPECIALITY HOSPITAL'
File '1664006393364_Discharge summary_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/FortuneUrologyClinic'
File '1664006429780_Patient Discharge Summary Form_438054_24-09-2022_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Gauhati Medical'
File '1664006472710_Patient Discharge Summary Form_23326_24-09-2022_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Daman'
File '1664007882043_DISCHARGESUMMARY_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Sri Aurobindo Institute of Medical Sciences'
File '1664007882043_DISCHARGESUMMARY_p2.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Sri Aurobindo Institute of Medical Sciences'
File '1664006960547_

File '1664010923868_Patient Discharge Summary Form_422243_24-09-2022_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Gauhati Medical'
File '1664011443781_DISCHARGESUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/SAIMS HOSPITAL'
File '1664011443781_DISCHARGESUMMARY_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/SAIMS HOSPITAL'
File '1664011443781_DISCHARGESUMMARY_p2.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/SAIMS HOSPITAL'
File '1664011443781_DISCHARGESUMMARY_p3.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/SAIMS HOSPITAL'
File '1664011150105_DISCHARGE SUMMARY0001_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/THE GUJARAT CANCER & RESEARCH INSTITUTE'
File '1664011173338_AB DISCHARGE SUMMARY.PDF_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/i

...

File '1664084622647_Hospital Discharge Summary1_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664084622647_Hospital Discharge Summary1_p2.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Mahavir Cardiac Hospital'
File '1664084622647_Hospital Discharge Summary1_p3.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664084622647_Hospital Discharge Summary1_p4.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664084670406_DISCHARGE SUMMARY0001_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/THE GUJARAT CANCER & RESEARCH INSTITUTE'
File '1664084887276_Hospital Discharge Summary_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospital

File '1664095101398_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/GNRC MEDICAL'
File '1664093644653_Patient Discharge Summary Form_2241334_25-09-2022_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/District hospital sidhi'
File '1664093673568_DISCHARGE SUMMARY _p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/CHIRAYU MEDICAL COLLEGE & HOSPITAL'
File '1664095758776_DISCHARGESUMMARY.PDF_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/OODLANDOSPITAL'
File '1664095763613_DISCHARGE SUMMARY.PDF_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/WOODLAND HOSPITAL'
File '1664095763613_DISCHARGE SUMMARY.PDF_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/WOODLAND HOSPITAL'
File '1664093696138_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/h

File '1664098358491_Hospital Discharge Summary_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664098358491_Hospital Discharge Summary_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664098358491_Hospital Discharge Summary_p2.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664098526533_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/GNRC MEDICAL'
File '1664252274099_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/CHIRAYU MEDICAL COLLEGE & HOSPITAI'
File '1664098558454_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/GNRC MEDICAL'
File '166

File '1664101643198_DISCHARGE SUMMARY_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/WOODLAND HOSPITAL'
File '1664101643198_DISCHARGE SUMMARY_p2.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/WOODLAND HOSPITAL'
File '1664101643198_DISCHARGE SUMMARY_p3.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/WOODLAND HOSPITAL'
File '1664101653849_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/GNRC MEDICAL'
File '1664101663236_Hospital Discharge Summary_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664101663236_Hospital Discharge Summary_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664101663236_Hospital Discharge Summary_p2.jpeg' copied to '

File '1664106376398_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Kurukshetra Nursing Home'
File '1664104123826_DISCHARGE SUMMARY_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/WOODLAND WK HOSPITAL'
File '1664104123826_DISCHARGE SUMMARY_p1.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/WOODLAND WK HOSPITAL'
File '1664104177657_Hospital Discharge Summary_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664104177657_Hospital Discharge Summary_p2.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Mahavir Cardiac Hospital'
File '1664104423571_Hospital Discharge Summary_p0.jpeg' copied to '/data/user/nevil/results/clustering/hospitalwise/images/by_name/Shri Bachubhai Dahyabhai Mehta Mahavir Heart Institute'
File '1664104423571_Hospital Disc